In [1]:
!pip install pymupdf nltk pandas gensim scikit-learn gradio langchain



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you 

In [2]:
import fitz
import re
import nltk
import pandas as pd
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Preprocessing

In [3]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words or t in ["ml","ai","cnn","rnn"]]
    return " ".join(tokens)

upload file

In [4]:
filename = "/content/Hands_On_Machine_Learning_with_Scikit_Learn_and_TensorFlow_Aurélien.pdf"
doc = fitz.open(filename)

# Extract pages as text
pages = [page.get_text().strip() for page in doc if page.get_text().strip()]

# Find the first page containing "Chapter 1" (case-insensitive)
start_page = 0
for i, p in enumerate(pages):
    if re.search(r"\bchapter\s*1\b", p, re.IGNORECASE):
        start_page = i
        break

# Keep only pages starting from Chapter 1
pages = pages[start_page:]

# Detect common headers/footers again but only from kept pages
headers = [p.split("\n")[0] for p in pages if p]
footers = [p.split("\n")[-1] for p in pages if p]
common_header = Counter(headers).most_common(1)[0][0]
common_footer = Counter(footers).most_common(1)[0][0]

# Remove headers/footers
clean_pages = []
for p in pages:
    lines = p.split("\n")
    lines = [line for line in lines if line not in [common_header, common_footer]]
    clean_pages.append(" ".join(lines))


# Chunk BEFORE preprocessing

In [8]:
full_text = " ".join(clean_pages)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)
chunks_texts = splitter.split_text(full_text)

# Map chunks to page numbers approximately
page_offsets = [len(p) for p in clean_pages]
chunks = []
char_idx = 0
page_idx = 0
for chunk in chunks_texts:
    while page_idx < len(page_offsets) and char_idx >= page_offsets[page_idx]:
        char_idx -= page_offsets[page_idx]
        page_idx += 1
    chunks.append((page_idx + 1, chunk))
    char_idx += len(chunk)

# Create DataFrame
df_chunks = pd.DataFrame(chunks, columns=["Page", "Text"])

# Explicitly load the punkt tokenizer
import nltk.data
nltk.data.load('tokenizers/punkt/english.pickle')

df_chunks["Processed_Text"] = df_chunks["Text"].apply(preprocess_text)
df_chunks.head()

,Page,Text,Processed_Text
0,1,CHAPTER 1 The Machine Learning Landscape When ...,chapter 1 machine learning landscape people he...
1,1,. It was followed by hundreds of ML applicatio...,followed hundred ml application quietly power ...
2,1,. Then we will look at the workflow of a typic...,look workflow typical ml project discus main c...
3,2,. What Is Machine Learning? Machine Learning i...,machine learning machine learning science art ...
4,2,".g., flagged by users) and examples of regular...",g flagged user example regular nonspam also ca...


# Train Word2Vec model

In [9]:
tokenized_chunks = [text.split() for text in df_chunks["Processed_Text"]]
w2v_model = Word2Vec(sentences=tokenized_chunks, vector_size=100, window=5, min_count=2, workers=4)


# Compute TF-IDF weighted embeddings

In [10]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_chunks["Processed_Text"])
feature_names = vectorizer.get_feature_names_out()

chunk_vectors = []
for i, tokens in enumerate(tokenized_chunks):
    vecs = []
    weights = []
    for token in tokens:
        if token in w2v_model.wv.key_to_index and token in vectorizer.vocabulary_:
            vecs.append(w2v_model.wv[token])
            weights.append(tfidf_matrix[i, vectorizer.vocabulary_[token]])
    if vecs:
        weighted_avg = np.average(vecs, axis=0, weights=weights)
    else:
        weighted_avg = np.zeros(w2v_model.vector_size)
    chunk_vectors.append(weighted_avg)
chunk_vectors = np.array(chunk_vectors)


# Define search function

In [11]:
def search_book(query, top_k=3):
    query_proc = preprocess_text(query)
    query_tokens = query_proc.split()
    vecs = []
    weights = []
    for token in query_tokens:
        if token in w2v_model.wv.key_to_index and token in vectorizer.vocabulary_:
            vecs.append(w2v_model.wv[token])
            weights.append(vectorizer.idf_[vectorizer.vocabulary_[token]])
    if vecs:
        query_vec = np.average(vecs, axis=0, weights=weights).reshape(1, -1)
    else:
        query_vec = np.zeros((1, w2v_model.vector_size))
    sims = cosine_similarity(query_vec, chunk_vectors).flatten()
    top_idx = sims.argsort()[::-1][:top_k]

    results = []
    for i in top_idx:
        page_num = df_chunks.iloc[i]["Page"]
        text = df_chunks.iloc[i]["Text"]
        score = sims[i]
        results.append([page_num, text[:500]+"...", round(score,3)])
    return results


# Launch Gradio interface

In [12]:
iface = gr.Interface(
    fn=search_book,
    inputs=[
        gr.Textbox(label="Enter your search query", placeholder="e.g. What is gradient descent?"),
        gr.Slider(1, 10, value=3, step=1, label="Number of results")
    ],
    outputs=gr.Dataframe(headers=["Page", "Text", "Score"]),
    title="📚 Machine Learning Book Search (Word2Vec)",
    description="Search the PDF document using Word2Vec embeddings."
)

iface.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2b8a0ef4ab9249e025.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2b8a0ef4ab9249e025.gradio.live
